https://nbviewer.jupyter.org/github/zgana/data-science-one-offs/blob/master/NLP-Analysis-of-The-Office.ipynb?flush_cache=true#Character-character-mentions

In [1]:
import pandas as pd

from schrutepy import schrutepy

In [2]:
MAIN_CHARACTERS = [
    "Michael", "Jim", "Pam", "Dwight", 
    "Phyllis", "Stanley", "Meredith",
    "Oscar", "Angela", "Kevin",
    "Ryan", "Toby", "Kelly", 
]

def subset_data(full_dataset, MAIN_CHARACTERS):
    subset = full_dataset[(full_dataset.character.isin(MAIN_CHARACTERS)) & (full_dataset.text.notna())]
    
    subset = subset.assign(
        keep = subset.apply(lambda x: True if any(c for c in MAIN_CHARACTERS if c in 
                                                  x.text) else False, axis=1)
    )

    df = subset[subset.keep==1].reset_index(drop=True)

    for c in MAIN_CHARACTERS:
        df[c] = df.apply(lambda x: 1 if c in x.text else 0, axis=1)
        
    return df

In [3]:
full_dataset = schrutepy.load_schrute()
df = subset_data(full_dataset, MAIN_CHARACTERS)

In [4]:
df.pivot_table(index="character", 
               values=MAIN_CHARACTERS, 
               aggfunc="sum").reset_index()

,character,Angela,Dwight,Jim,Kelly,Kevin,Meredith,Michael,Oscar,Pam,Phyllis,Ryan,Stanley,Toby
0,Angela,9,73,11,8,38,14,28,26,29,16,1,2,7
1,Dwight,55,97,272,20,41,22,264,27,98,45,41,43,22
2,Jim,22,243,45,21,18,15,177,15,197,32,20,23,15
3,Kelly,2,16,21,9,5,6,25,4,14,0,44,1,8
4,Kevin,24,13,38,3,24,5,72,37,41,8,6,2,4
5,Meredith,1,2,9,1,0,10,8,3,9,0,0,1,3
6,Michael,47,330,220,29,86,62,158,92,302,114,138,124,99
7,Oscar,43,25,18,5,34,6,71,7,24,3,5,7,9
8,Pam,47,127,185,14,34,21,268,25,59,29,32,18,18
9,Phyllis,14,35,16,1,2,2,61,2,12,1,3,12,3
